# 저장된 모델을 활용한 온실 환경 예측 및 결과 생성
이 노트북은 학습된 XGBoost 모델을 불러와 테스트 데이터에 대해 예측을 수행하고, 결과를 sample_submission 형식으로 저장합니다.

In [16]:
import pandas as pd
import numpy as np
import joblib

In [17]:
# 테스트 데이터 및 제출 파일 로드
test_X = pd.read_csv('test_X.csv')
sample_submission = pd.read_csv('sample_submission.csv')
test_X['time'] = pd.to_datetime(test_X['time'])

In [18]:
# 피쳐 엔지니어링 함수 정의 (학습과 동일하게 적용)
def clean_data(df):
    df_clean = df.copy()
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='both')
        df_clean[col] = df_clean[col].fillna(0)
    return df_clean

def create_time_features(df):
    df_new = df.copy()
    df_new['hour'] = df_new['time'].dt.hour
    df_new['day_of_week'] = df_new['time'].dt.dayofweek
    df_new['month'] = df_new['time'].dt.month
    df_new['is_weekend'] = (df_new['time'].dt.dayofweek >= 5).astype(int)
    df_new['hour_sin'] = np.sin(2 * np.pi * df_new['hour'] / 24)
    df_new['hour_cos'] = np.cos(2 * np.pi * df_new['hour'] / 24)
    df_new['month_sin'] = np.sin(2 * np.pi * df_new['month'] / 12)
    df_new['month_cos'] = np.cos(2 * np.pi * df_new['month'] / 12)
    return df_new

def create_interaction_features(df):
    df_new = df.copy()
    interactions = [
        ('temperature_outside', 'humidity_outside', 'temp_humid_interact'),
        ('solar_radiation', 'temperature_outside', 'solar_temp_interact'),
        ('greenhouse_roof_vent1', 'greenhouse_roof_vent2', 'vent_total'),
        ('curtain1', 'curtain2', 'curtain_avg'),
    ]
    for col1, col2, new_name in interactions:
        if col1 in df_new.columns and col2 in df_new.columns:
            if 'total' in new_name:
                df_new[new_name] = df_new[col1] + df_new[col2]
            elif 'avg' in new_name:
                df_new[new_name] = (df_new[col1] + df_new[col2]) / 2
            else:
                df_new[new_name] = df_new[col1] * df_new[col2]
    return df_new

In [19]:
# 테스트 데이터 피쳐 엔지니어링
test_clean = clean_data(test_X)
test_proc = create_time_features(test_clean)
test_proc = create_interaction_features(test_proc)

In [20]:
# 학습에서 사용한 feature 컬럼 정의 (train_pipeline과 동일하게 맞춰야 함)
feature_cols = [col for col in test_proc.columns if col not in ['time']]

In [21]:
# 저장된 모델 불러와서 예측 수행 (각 타겟별 5개 모델 앙상블 평균)
targets = ['temperature', 'humidity', 'co2']
preds = {target: np.zeros(len(test_proc)) for target in targets}
for target in targets:
    fold_preds = []
    for fold in range(5):
        model = joblib.load(f'model-{target}-{fold}.pkl')
        fold_pred = model.predict(test_proc[feature_cols])
        fold_preds.append(fold_pred)
    preds[target] = np.mean(fold_preds, axis=0)

In [22]:
# sample_submission 파일에 예측값 입력 및 저장
for target in targets:
    # sample_submission의 time에 맞는 인덱스만 추출
    idx = test_proc['time'].isin(sample_submission['time'])
    sample_submission[target] = preds[target][idx]
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

/var/folders/66/5s9xg91x36d70dj68vy37x340000gn/T/ipykernel_21704/3019156117.py:4: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  idx = test_proc['time'].isin(sample_submission['time'])


,time,temperature,humidity,co2
0,2024-06-25 00:00:00,16.775021,85.792130,362.912170
1,2024-06-25 00:05:00,16.822609,86.507500,363.626373
2,2024-06-25 00:10:00,16.806879,86.042519,363.899750
3,2024-06-25 00:15:00,16.768024,86.602615,363.899750
4,2024-06-25 00:20:00,16.728352,86.164696,363.899750
